In [1]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [2]:
dataset = pd.read_csv('../Datasets/Thyroid/thyroidDF.csv')

In [3]:
diagnoses = {'-': 'negative',
             'A': 'hyperthyroid', 
             'C': 'hyperthyroid', 
             'B': 'hyperthyroid', 
             'D': 'hyperthyroid',
             'E': 'hypothyroid', 
             'F': 'hypothyroid', 
             'G': 'hypothyroid', 
             'H': 'hypothyroid'}

dataset['target'] = dataset['target'].map(diagnoses) # re-mapping
dataset.dropna(subset=['target'], inplace=True)

In [4]:
dataset.drop(['TSH_measured', 'T3_measured', 'TT4_measured', 'T4U_measured', 'FTI_measured', 'TBG_measured', 'patient_id', 'referral_source', 'TBG'], axis=1, inplace=True)

In [5]:
dataset[dataset['age']>100]

dataset.drop(dataset[dataset['age']>100].index, inplace=True)

In [6]:
dataset['sex'] = np.where((dataset.sex.isnull()) & (dataset.pregnant == 't'), 'F', dataset.sex)

In [7]:
dataset.drop('T3', axis=1, inplace=True)

In [8]:
dataset.dropna(inplace=True)

In [9]:
dataset.replace('f', 0, inplace=True)
dataset.replace('t', 1, inplace=True)
dataset.replace('M', 0, inplace=True)
dataset.replace('F', 1, inplace=True)

In [10]:
diagnoses = {'negative': 0,
             'hypothyroid': 1, 
             'hyperthyroid': 2}

dataset['target'] = dataset['target'].map(diagnoses) # re-mapping

# train and test split --> stratified
X = dataset.drop('target', axis=1).copy()
y = dataset['target'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# TRAIN MODEL

In [11]:
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=120)
automl.fit(X_train.values, y_train)
y_hat = automl.predict(X_test.values)

In [12]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.9839743589743589

In [13]:
import pickle
with open('./models/thyroid_automl.pkl', 'wb') as f:
    pickle.dump(automl, f)

# LOAD MODEL

In [11]:
import pickle
with open('./models/thyroid_automl.pkl', 'rb') as f:
    automl = pickle.load(f)

In [12]:
len(automl.show_models())

8

# LIME

In [13]:
import lime
import lime.lime_tabular
import tqdm 

In [14]:
continuous_features = ['age', 'TSH', 'TT4', 'T4U', 'FTI']
categorical_features = X_train.columns.drop(continuous_features).tolist()
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.tolist(), class_names=['negative', 'hypothyroid', 'hyperthyroid'], categorical_features=categorical_features, discretize_continuous=True)

In [15]:
text_x = X_test.values

In [16]:
exp_fn = lambda i: explainer.explain_instance(X_test.iloc[i], automl.predict_proba, num_features=len(X_test.columns))
def exp_fn_blk(xtest, exp_fn):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = exp_fn(i)
        exp1.append(exp.as_map()[exp.available_labels()[0]])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x, exp_fn))

In [17]:
import metrics

In [ ]:
exp1 = exp_fn_wrap(text_x)
exp2 = exp_fn_wrap(text_x)

In [19]:
np.save('./explanations/thyroid1.npy', exp1)
np.save('./explanations/thyroid2.npy', exp2)

In [18]:
exp1 = np.load('./explanations/thyroid1.npy')
exp2 = np.load('./explanations/thyroid2.npy')

In [19]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [20]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(X_test.columns))
sb = metrics.calc_stability(enc1, y_test)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [21]:
i, s, sb

((100.0, 0, 1560), (0, 1560, 2433600, 0.0), (138, 1560))

In [22]:
X_test_norm = metrics.normalize_test(X_train, X_test)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [23]:
sim

0.1767860825689674

In [24]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in tqdm.tqdm(range(len(text_x))):
    atr = exp1[i]
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(3, dtype=int)
    np.put(y, y_test.iloc[i], 1)
    example = metrics.FeatureAttribution(automl, text_x[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

100%|██████████| 1560/1560 [1:51:13<00:00,  4.28s/it]


In [25]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))

-0.14397518127304595
1.0
0.0
-0.16240601503759397
1.0
0.0


In [26]:
metrics.calc_trust_score(automl, text_x, exp1, 3, X_train.columns.to_list())

100%|██████████| 1560/1560 [5:29:17<00:00, 12.67s/it]  


0.2674145299145299

# CIU

In [13]:
from ciu import determine_ciu
import tqdm
import metrics

In [14]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [15]:
feat_list = X_train.columns.tolist()

In [16]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = determine_ciu(X_test.iloc[i:i+1], automl.predict_proba, X_train.to_dict('list'), samples = 1000, prediction_index = 1)
        exp_list = [[feat_list.index(i), exp.ci[i]] for i in exp.ci]
        exp1.append(exp_list)
    return np.array(exp1)

In [31]:
exp1 = exp_fn_blk(X_test)
exp2 = exp_fn_blk(X_test)

  0%|          | 0/1560 [00:00<?, ?it/s]

100%|██████████| 1560/1560 [1:20:04<00:00,  3.08s/it]


In [32]:
np.save('./explanations/thyroid_ciu1.npy', exp1)
np.save('./explanations/thyroid_ciu2.npy', exp2)

In [17]:
exp1 = np.load('./explanations/thyroid_ciu1.npy')
exp2 = np.load('./explanations/thyroid_ciu2.npy')

In [33]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(feat_list))
sb = metrics.calc_stability(enc1, y_test)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [34]:
i, s, sb

((100.0, 0, 1560), (0, 1560, 2433600, 0.0), (-659, 1560))

In [35]:
X_test_norm = metrics.normalize_test(X_train, X_test)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [36]:
sim

0.0996306921268756

In [38]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in tqdm.tqdm(range(len(X_test))):
    atr = exp1[i]   
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(3, dtype=int)
    np.put(y, y_test.iloc[i], 1)
    example = metrics.FeatureAttribution(automl, X_test.to_numpy()[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

100%|██████████| 1560/1560 [2:08:29<00:00,  4.94s/it] 


In [39]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))

0.5781865931413919
0.9993589743589744
0.0
0.6314404283531938
1.0
0.0


In [18]:
metrics.calc_trust_score(automl, X_test.to_numpy(), exp1, 3, X_train.columns.to_list())

100%|██████████| 1560/1560 [5:50:29<00:00, 13.48s/it]  


0.28237179487179487

# RULEFIT

In [13]:
from skrules import SkopeRules
import metrics_rules
import time

In [14]:
clf = SkopeRules(max_depth_duplication=2,
                    n_estimators=512,
                    precision_min=0.3,
                    recall_min=0.1,
                    feature_names=X_train.columns.tolist())

In [15]:
start_time = time.time()
clf.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/skrules/skope_rules.py:220: UserWarning: Found labels {0, 1, 2}. This method assumes target class to be labeled as 1 and normal data to be labeled as 0. Any label different from 0 will be considered as being from the target class.
  % set(self.classes_))


--- 46.34715509414673 seconds ---


In [16]:
start_time = time.time()
top_rules1 = clf.score_top_rules(X_test)
top_rules2 = clf.score_top_rules(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.06779360771179199 seconds ---


In [17]:
i = metrics_rules.calc_identity_rules(top_rules1, top_rules2)
print(i)

s = metrics_rules.calc_separability_rules(top_rules1)
print(s)

enc_rules = metrics_rules.exp_enc(clf, top_rules1)
sb = metrics_rules.calc_stability_rules(enc_rules, y_test)
print(sb)

(0.0, 1560, 1560)


(1928974, 1560, 2433600, 79.26421761998685)
(50, 1560)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [18]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test)
sim = metrics_rules.calc_similarity(enc_rules, X_test_norm)
print(sim)

0.0


# RULEMATRIX

In [13]:
continuous_features = ['age', 'TSH', 'TT4', 'T4U', 'FTI']
categorical_features = X_train.columns.drop(continuous_features).tolist()

In [14]:
import rulematrix
import time
import metrics_rules

In [15]:
is_continuous = [True if i in continuous_features else False for i in X_train.columns.tolist()]
is_categorical = [True if i in categorical_features else False for i in X_train.columns.tolist()]

In [16]:
surrogate = rulematrix.surrogate.rule_surrogate(
    automl.predict,
    X_train,
    sampling_rate=4,
    is_continuous=is_continuous,
    is_categorical=is_categorical,
    seed=42
)

In [17]:
test_x = X_test.to_numpy()

In [18]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in range(len(xtest)):
        queried_rules = np.arange(surrogate.student.n_rules)[surrogate.student.decision_path(test_x[i].reshape(1,-1)).reshape(-1)]
        exp1.append(queried_rules[-1])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x))

In [19]:
start_time = time.time()
exp1 = exp_fn_blk(test_x)
exp2 = exp_fn_blk(test_x)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.8688592910766602 seconds ---


In [20]:
def enc_exp(exp, n_features):
    enc = []
    for i in range(exp.shape[0]):
        new = np.zeros(n_features)
        for j in surrogate.student.rule_list[exp[i]].clauses:
            new[j.feature_idx] = 1
        enc.append(new)
    return np.array(enc)

In [21]:
enc_exp = enc_exp(exp1, X_train.shape[1])

In [22]:
i = metrics_rules.calc_identity_rules(exp1, exp2)
print(i)

s = metrics_rules.calc_separability_rules(exp1)
print(s)

sb = metrics_rules.calc_stability_rules(enc_exp, y_test)
print(sb)

(0.0, 1560, 1560)
(1265948, 1560, 2433600, 52.01955950032873)
(101, 1560)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [23]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test)
sim = metrics_rules.calc_similarity(enc_exp, X_test_norm)

In [24]:
sim

0.0